# Aufgabe 2: Data Exploration und Analyse

In [ ]:
import pandas as pd

In [ ]:
# dataset = pd.read_csv('/content/drive/My Drive/Datascience/DatenAusgegeben1.0.csv', delimiter=';')
# Datensatz einlesen
dataset = pd.read_csv('./DatenAusgegeben1.0.csv', delimiter=';', encoding = 'cp852')
dataset

In [ ]:
#Import seaborn und pyplot
import seaborn as sb
import matplotlib.pyplot as plt

sb.pairplot(data = dataset)
plt.show()

In [ ]:
#dataset_noprice = dataset[dataset.columns.difference(['Preis', 'Grundstück in qm'])]
plt.figure(figsize = (10, 10))

#sb.boxplot(data = dataset_noprice)
columns = ['Grundstück in qm', 'Zustand', 'Gebaut',
           'Renoviert', 'Zustand Fassade', 'Kellerflche in qm',
           'Erster Stock in qm', 'Zweiter Stock in qm', 'Wohnflche in qm', 
           'Schlafzimmer', 'Küchen', 'Küchenqualitt', 'Rume', 'Garagenkapazitt',
           'Verkaufsmonat', 'Verkaufsjahr', 'Preis']
for column in columns:
  print(column)
  sb.boxenplot(data = dataset[column])
  plt.show()
  # pd.plotting.boxplot(data = dataset[[column]])

Zuerst müssen Anomalien im Datensatz gefunden werden, hierfür eigenen sich Boxplots sehr gut. Dabei können ähnliche Attribute (z.B Jahreszahlen, oder Flächenangaben) in eine Figur gezeichnet werden:

In [ ]:
#sb.boxplot(data = dataset_noprice)
import math

fig, axes = plt.subplots(4, 5, figsize = (10, 15))
i = 0
plt.tight_layout(pad = 2)
for column in columns:
  chosenax = axes[math.floor(i / 5)][i % 5]
  chosenax.set_title(column)
  sb.boxplot(data = dataset[column], ax = chosenax)
  i = i + 1
plt.show()
  # pd.plotting.boxplot(data = dataset[[column]])

#TODO: Anomalien beschreiben

In [ ]:
sb.boxenplot(data = dataset[['Verkaufsjahr', 'Preis']], x = 'Verkaufsjahr', y= 'Preis')
plt.show()

In [ ]:
import math

attribute_columns = ['Grundstücksform', 'Steigung', 'Zone', 'Lage',
'Typ', 'Zustand', 'Zustand Fassade', 'Heizung',
'Heizungsqualitt', 'Klimaanlage', 'Schlafzimmer', 'Küchen',
'Küchenqualitt', 'Rume', 'Garage Typ', 'Garagenkapazitt', 'Pool']

data_no_nan = dataset[attribute_columns].fillna('Nix')

column_amount = len(data_no_nan.columns)
fig, axes = plt.subplots(math.ceil(column_amount / 4), 4, figsize = (25, 15))

i = 0
plt.tight_layout(pad = 3)
for column in data_no_nan.columns:
  chosenax = axes[math.floor(i / 4)][i % 4]
  chosenax.set_title(column)
  sb.countplot(data = data_no_nan, x = column, ax = chosenax)
  i = i + 1
plt.show()


In [ ]:
plt.figure(figsize = (15,15))
sb.heatmap(dataset.fillna(0).corr(), annot = True, cmap = 'viridis')
plt.show()

In [ ]:

plt.figure(figsize = (20, 10))
sb.heatmap(data = dataset.fillna(0).corr()['Preis'].to_frame(), annot = True, cmap = 'viridis')
plt.show()

Wichtig könnte sein:
- Gebaut ?
- Renoviert
- Kellerfläche
- Wohnfläche
- Küchenqualtiät
- Räume
- Garagenkapazität

In [ ]:
sb.boxenplot(data = dataset, x = 'Küchenqualitt', y = 'Preis')

In [ ]:
sb.boxenplot(data = dataset, x = 'Zustand', y = 'Preis')